In [16]:
!pip install transformers

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
     ---------------------------------------- 3.3/3.3 MB 5.2 MB/s eta 0:00:00
     -------------------------------------- 895.2/895.2 KB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 77.9/77.9 KB ? eta 0:00:00
     -------------------------------------- 40.8/40.8 KB 649.1 kB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:
      Successfully uninstalled typing-extensions-3.7.4.2
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4


In [28]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizerFast, BertModel

In [31]:
class TweetDataset(Dataset):
    def __init__(self, data_type, max_seq_len):
        self.max_seq_len = max_seq_len
        # read pre-processed data
        self.tweet_df = pd.read_csv(f'{data_type}_tweet_df.csv', usecols=['text', 'reply_text', 'label'])
        self.statistic_df = pd.read_csv(f'{data_type}_stat_feat_df.csv')
        self.tweet_df['text'] = self.tweet_df['text'].replace(np.nan, '')
        self.tweet_df['reply_text'] = self.tweet_df['reply_text'].replace(np.nan, '')
        # define tokenizer
        self.tokenizer = DistilBertTokenizerFast.from_pretrained("bert-base-uncased")
    def __len__(self):
        return self.tweet_df.shape[0]
    def __getitem__(self, idx):
        source_token_mask = self.tokenizer(self.tweet_df.iloc[idx]['text'], truncation=True, padding='max_length', max_length=self.max_seq_len)
        source_token, source_mask = torch.tensor(source_token_mask['input_ids']), torch.tensor(source_token_mask['attention_mask'])
        pair_token_mask = self.tokenizer(self.tweet_df.iloc[idx]['text'], self.tweet_df.iloc[idx]['reply_text'], truncation='only_second', padding='max_length', max_length=self.max_seq_len)
        pair_tokens_tensor, pair_mask_tensor = torch.tensor(pair_token_mask['input_ids']), torch.tensor(pair_token_mask['attention_mask'])
        return source_token, source_mask, pair_tokens_tensor, pair_mask_tensor, self.tweet_df.iloc[idx]['label'], torch.tensor(self.statistic_df.iloc[idx])

In [32]:
train_loader = DataLoader(TweetDataset('train', 200), shuffle=True, batch_size=20, drop_last=True)
dev_loader = DataLoader(TweetDataset('dev', 200), shuffle=True, batch_size=20, drop_last=True)

FileNotFoundError: [Errno 2] File train_tweet_df.csv does not exist: 'train_tweet_df.csv'